In [140]:
import json
import pathlib

package_paths = {
    "ips": pathlib.Path(".") / "packages" / "hl7.fhir.uv.ips-1.1.0",
    "zib2017": pathlib.Path(".") / "packages" / "nictiz.fhir.nl.stu3.zib2017-2.2.20",
    "zib2020": pathlib.Path(".") / "packages" / "nictiz.fhir.nl.r4.zib2020-0.11.0-beta.1"
}

class Profile:
    def __init__(self, package, profile_name):
        self.package = package
        self.name = profile_name

        content = self._open(self)
        
        # Fish out interesting data: min and max cardinality and binding strengths
        self.of_interest = [el["path"] for el in content["differential"]["element"] if "min" in el or "max" in el or "binding" in el]

    def fits_in(self, other):
        self_snap = self._open(self)["snapshot"]["element"]
        other_snap = self._open(other)["snapshot"]["element"]

        findings = []
        # If we want to know if we "fit in" the other, we have to look at the restrictions done by the other and see
        # if we're compatible. We don't need to look at our own restrictions
        for path in other.of_interest:
            try:
                self_el = list(filter(lambda e: e["path"] == path, self_snap))[0]
                other_el = list(filter(lambda e: e["path"] == path, other_snap))[0]
            except IndexError:
                # Happens when we have extensions. Need to look into this
                continue

            if other_el["min"] > self_el["min"]:
                findings.append(f"{path}: required in {other.name} but optional in {self.name}.")

            if other_el["max"] != "*": # if other is unrestricted we always fit
                if self_el["max"] == "*":
                     findings.append(f"{path}: restricted to {other_el["max"]} in {other.name} but unrestricted in {self.name}.")
                else:
                    self_max = int(self_el["max"])
                    other_max = int(other_el["max"])
                    if self_max > other_max:
                        findings.append(f"{path}: restricted to {other_max} in {other.name} but to {self_max} in {self.name}.")

            if "binding" in other_el:
                self_vs = self._getBoundValueSet(self_el)
                other_vs = self._getBoundValueSet(other_el)
                if other_el["binding"]["strength"] == "required":
                    if self_vs != other_vs:
                        findings.append(f"{path}: required Valueset in {other.name} differs from {self.name}. Manual check needed.")
                    elif self_el["binding"]["strength"] != "required":
                        findings.append(f"{path}: required binding in {other.name} is more restrictive than {self_el["binding"]["strength"]} binding in {self.name}.")
                elif other_el["binding"]["strength"] == "extensible":
                    if self_vs != other_vs:
                        findings.append(f"{path}: required Valueset in {other.name} differs from {self.name}. Manual check needed.")
                    elif self_el["binding"]["strength"] not in ["extensbile", "required"]:
                        findings.append(f"{path}: extensible ValueSet in {other.name} is more restrictive than {self_el["binding"]["strength"]} binding in {self.name}.")
                else:
                    if self_vs != other_vs:
                        findings.append(f"{path}: differences in Valueset preferences. Manual check needed.")
                    else:
                        # We don't care much about other differences
                        pass
                        
        if len(findings):
            print(f"{self.name} ({self.package}) does not seem to fit in {other.name} ({other.package})"),
            for finding in findings:
                print(f"* {finding}")

    def _open(self, profile):
        if profile.package == "ips":
            file = package_paths[profile.package] / f"StructureDefinition-{profile.name}.json"
        else:
            file = package_paths[profile.package] / f"{profile.name}.json"

        with open(file) as fd:
            content = json.load(fd)

        return content

    def _getBoundValueSet(self, snapshot_element):
        if "valueSetReference" in snapshot_element["binding"]:
            return snapshot_element["binding"]["valueSetReference"]["reference"]
        return snapshot_element["binding"]["valueSet"]

zib_Problem2017 = Profile("zib2017", "zib-Problem")
zib_Problem2020 = Profile("zib2020", "zib-Problem")
Condition_uv_ips = Profile("ips", "Condition-uv-ips")
zib_Problem2017.fits_in(Condition_uv_ips)
Condition_uv_ips.fits_in(zib_Problem2017)
zib_Problem2020.fits_in(Condition_uv_ips)
Condition_uv_ips.fits_in(zib_Problem2020)

zib-Problem (zib2017) does not seem to fit in Condition-uv-ips (ips)
* Condition.clinicalStatus: required Valueset in Condition-uv-ips differs from zib-Problem. Manual check needed.
* Condition.category: differences in Valueset preferences. Manual check needed.
* Condition.code: differences in Valueset preferences. Manual check needed.
* Condition.bodySite: differences in Valueset preferences. Manual check needed.
Condition-uv-ips (ips) does not seem to fit in zib-Problem (zib2017)
* Condition.clinicalStatus: required Valueset in zib-Problem differs from Condition-uv-ips. Manual check needed.
* Condition.verificationStatus: required Valueset in zib-Problem differs from Condition-uv-ips. Manual check needed.
* Condition.category: required Valueset in zib-Problem differs from Condition-uv-ips. Manual check needed.
* Condition.code: required Valueset in zib-Problem differs from Condition-uv-ips. Manual check needed.
* Condition.bodySite: required Valueset in zib-Problem differs from Condi